In [1]:
from numpy.polynomial import Chebyshev
import numpy as np
import sympy as sp
x = sp.Symbol('x')


In [2]:
import numpy as np
import sympy as sp

x = sp.Symbol('x', real=True)
k = sp.Symbol('k', integer=True, positive=True)

Tk = lambda k, x: sp.cos(k * sp.acos(x))
cj = lambda j: 2 if j == 0 else 1

def innerw(u, v, domain, ref_domain=(-1, 1)):
    A, B = ref_domain
    a, b = domain
    # map u(x(X)) to use reference coordinate X.
    # Note that x is here ref coord.
    us = u.subs(x, a + (b-a)*(x-A)/(B-A))
    # Change variables x=cos(theta)
    us = sp.simplify(us.subs(x, sp.cos(x)), inverse=True)
    vs = sp.simplify(v.subs(x, sp.cos(x)), inverse=True)
    return sp.integrate(us*vs, (x, 0, sp.pi))


In [3]:
from scipy.integrate import quad
def innerwn(u, v, domain, ref_domain=(-1, 1)):
    A, B = ref_domain
    a, b = domain
    us = u.subs(x, a + (b-a)*(x-A)/(B-A))
    us = sp.simplify(us.subs(x, sp.cos(x)), inverse=True)
    vs = sp.simplify(v.subs(x, sp.cos(x)), inverse=True)
    return quad(sp.lambdify(x, us*vs), 0, np.pi)[0]

u = sp.exp(sp.cos(x))
#u = 1 / (1+16*x**2)
#uhat = lambda u, j: 2 / (cj(j) * sp.pi) * innerw(u, Tk(j, x), (-1, 1)) # slow
uhatn = lambda u, j: 2 / (cj(j) * np.pi) * innerwn(u, Tk(j, x), (-1, 1))

In [4]:
uc = []
N = 25
for n in range(N):
    uc.append(uhatn(u, n))

In [ ]:
def L2_error(uh, ue, space):
    xj = np.linspace(-1, 1, 1000)
    uej = sp.lambdify(x, ue)(xj)
    err = []
    for n in range(0, len(uh), 2):
        uj = space(uh[:(n+1)])(xj).astype(float)
        err.append(np.sqrt(np.trapz((uj-uej)**2, dx=xj[1]-xj[0])))
    return err

ue = sp.besselj(0, x)
errc = L2_error(uc, u, Chebyshev)
errc

In [ ]:
def L2_error_glerkin(uh, ue, V, kind='norm'):
    d = V.domain
    uej = sp.lambdify(x, ue)
    #def uv(xj): return (uej(xj)-V.eval(uh, xj))**2
        
    if kind == 'norm':
        return np.sqrt(quad(uv, float(d[0]), float(d[1]))[0])
    #elif kind == 'inf':
     #   return max(abs(uj-uej))

In [9]:
ue = sp.besselj(0, x)
uej = sp.lambdify(x, ue)
uej

<function _lambdifygenerated(x)>

In [ ]:
def uv(xj): 
    return (uej(xj)-V.eval(uh, xj))**2

In [8]:
#ue - exact
#uh aprozimated
def L2_error(uh, ue, space):
    xj = np.linspace(-1, 1, 1000)
    uej = sp.lambdify(x, ue)(xj)
    err = []
    for n in range(0, len(uh), 2):
        uj = space(uh[:(n+1)])(xj).astype(float)
        err.append(np.sqrt(np.trapz((uj-uej)**2, dx=xj[1]-xj[0])))
    return err

ue = sp.besselj(0, x)

errc = L2_error(uc, u, Chebyshev)
errc

[0.4974667007690379,
 0.03988279805115948,
 0.00245173162420416,
 0.00012792744536135574,
 5.8957014692144055e-06,
 2.4627595493254023e-07,
 9.484340797012685e-09,
 3.408942020425187e-10,
 1.154143058382014e-11,
 3.7091565443472394e-13,
 1.1604118055634894e-14,
 2.7118106349134324e-15,
 4.03066014213448e-15]

In [ ]:
err < 1e-6

from numpy.polynomial import Chebyshev as Cheb

class Chebyshev2(FunctionSpace):
    def __init__(self, N, domain=(-1, 1)):
        FunctionSpace.__init__(self, N, domain=domain)
        self._cheb_basis = [Cheb.basis(i, domain=domain) for i in range(N + 1)]

    def basis_function(self, j, sympy=False):
        if sympy:
            return sp.Chebyshev.basis(j, domain=self._domain)
        return self._cheb_basis[j]

    def mass_matrix(self):
        M = np.zeros((self.N + 1, self.N + 1))

        for i in range(self.N + 1):
            for j in range(self.N + 1):
                # Calculate the inner product using the weight function
                M[i, j] = np.sum(self._cheb_basis[i] * self._cheb_basis[j])

        return M

    def eval(self, uh, xj):
        xj = np.atleast_1d(xj)
        Xj = map_reference_domain(xj, self.domain, self._reference_domain)
        return np.polynomial.chebyshev.chebval(Xj, uh)

def L2_error(uh, ue, V, kind='norm'):
    d = V.domain
    uej = sp.lambdify(x, ue)
    def uv(xj): return (uej(xj)-V.eval(uh, xj))**2
    if kind == 'norm':
        return np.sqrt(quad(uv, float(d[0]), float(d[1]))[0])
    elif kind == 'inf':
        return max(abs(uj-uej))

ue = sp.besselj(0, x)
domain = (0, 10)
cheb = Chebyshev2(16, domain)
cheb